In [1]:
import pandas as pd
import sqlite3
import numpy as np
import copy
from tf.app import use
from IPython.display import display, HTML

pd.set_option('display.max_columns', None)

In [3]:
TOTHT_FILE = '../data_files/STEP/TOTHT.csv'
TOTHT_DF = pd.read_csv(TOTHT_FILE, sep='\t', low_memory=False)

In [150]:
TBESH_FILE = '../data_files/STEP/TBESH.csv'
TBESH_DF = pd.read_csv(TBESH_FILE, sep='\t', low_memory=False, index_col=False)

In [151]:
print(TBESH_DF.iloc[0]['lex'])

אָב


In [152]:
display(HTML(TBESH_DF.head(n=2).to_html(index=False)))

strongs,lex,transliteration,morph,gloss,definition
H0001,אָב,av,H:N-M,father,"1) father of an individual<br>2) of God as father of his people<br>3) head or founder of a household, group, family, or clan<br>4) ancestor<br>4a) grandfather, forefathers - of person<br>4b) of people<br>5) originator or patron of a class, profession, or art<br>6) of producer, generator (fig.)<br>7) of benevolence and protection (fig.)<br>8) term of respect and honour<br>9) ruler or chief (spec.)<br>"
H0002,אַב,av,A:N-M,father,1) father<br>


In [155]:
tb_lex = [l for l in TBESH_DF['lex']]
lex2 = set([i for i in OHB_ALIGNED['lex']])
tb_str = [s for s in TBESH_DF['strongs']]
lex_map = {tb_lex[i]:tb_str[i] for i in range(len(tb_lex))}

In [ ]:
lex_map = {tb_lex[i]:tb_str[i] for i in range(len(tb_lex))}
for l in lex2:
    if l not in lex_map:
        print(l)
        

In [4]:
display(HTML(TBESH_DF.head(n=7).to_html(index=False)))

Ref in Heb,Eng ref,Pointed,Accented,Morphology,Extended Strongs
Gen.1.1-01,Gen.1.1-01,בְּרֵאשִׁית,בְּ/רֵאשִׁ֖ית,HR/Ncfsa,H9003=ב=in/H7225=רֵאשִׁית=first_§1_beginning
Gen.1.1-02,Gen.1.1-02,בָּרָא,בָּרָ֣א,HVqp3ms,H1254a=בָּרָא=to create
Gen.1.1-03,Gen.1.1-03,אֱלֹהִים,אֱלֹהִ֑ים,HNcmpa,H0430=אֱלֹהִים=God_§God@Gen.1.1
Gen.1.1-04,Gen.1.1-04,אֵת,אֵ֥ת,HTo,H0853=אֵת=obj.
Gen.1.1-05,Gen.1.1-05,הַשָּׁמַיִם,הַ/שָּׁמַ֖יִם,HTd/Ncmpa,H9009=ה=the/H8064=שָׁמַיִם=heaven
Gen.1.1-06,Gen.1.1-06,וְאֵת,וְ/אֵ֥ת,HC/To,H9002=ו=and/H0853=אֵת=obj.
Gen.1.1-07,Gen.1.1-07,הָאָרֶץ,הָ/אָֽרֶץ/׃,HTd/Ncbsa,H9009=ה=the/H0776=אֶ֫רֶץ=land_§3_planet/H9016=׃=verseEnd


In [ ]:
i = 595
display(HTML(TBESH_DF.loc[i:i+40].to_html(index=False)))

In [31]:
print(TBESH_DF.iloc[6]['Accented'].split('/')[-1])

׃


In [42]:
print(max([len(i.split('/')) for i in morph if type(i) == str]))

5


In [32]:
# Word count for the dataset.
morph = [i for i in TBESH_DF['Morphology']]
count = 0
suffix = 0
for i in range(len(morph)):
    count +=1
    if type(morph[i]) == float:
        pass
    else:
        data = morph[i].split('/')
        if data[-1][0] in ['S']:
            if len(data) == 1:
                print(data)
            count += len(data)-2
            suffix += 1
        else:
            count += len(data)-1
print(count, suffix)

424014 47377


In [89]:
data = {'ref':[], 'word':[], 'lex':[], 'morph':[], 'strong':[], 'suffix':[], 'suffix_strong':[]}
refs = [i for i in TBESH_DF['Ref in Heb']]
words = [i for i in TBESH_DF['Accented']]
morphs = [i for i in TBESH_DF['Morphology']]
strongs = [i for i in TBESH_DF['Extended Strongs']]
for i in range(len(refs)):
    if type(words[i]) != str:
        for k in data:
            data[k].append(None)
    else:
        morph = morphs[i].split('/')
        count = len(morph) 
        strong_data = strongs[i].split('/')
        strong = []
        lex = []
        word = words[i].split('/')
        if count > 1:
            for s in strong_data:
                sd = s.split('=')
                if sd[0] not in ['_', '', ' ']:
                    # print(i, sd)
                    strong.append(sd[0])
                    try:
                        lex.append(sd[1])
                    except IndexError:
                        print(i, sd)
                else:
                    strong.append('')
                    lex.append('')
            if morph[-1][0] in ['S']:
                for j, m in enumerate(morph[:-2]):
                    data['ref'].append(refs[i])
                    data['word'].append(word[j])
                    data['lex'].append(lex[j])
                    data['morph'].append(m)
                    data['suffix'].append(None)
                    data['strong'].append(strong[j])
                    data['suffix_strong'].append(None)
                data['ref'].append(refs[i])
                data['word'].append(word[count-2]+word[count-1])
                data['lex'].append(lex[count-2])
                data['morph'].append(morph[-2])
                data['suffix'].append(word[count-1])
                data['strong'].append(strong[count-2])
                data['suffix_strong'].append(strong[count-1])
            else:
                for j, m in enumerate(morph):
                    data['ref'].append(refs[i])
                    data['word'].append(word[j])
                    data['lex'].append(lex[j])
                    data['morph'].append(m)
                    data['suffix'].append(None)
                    data['strong'].append(strong[j])
                    data['suffix_strong'].append(None)
        else:
            sd = strong_data[0].split('=')
            strong.append(sd[0])
            lex.append(sd[1])
            data['ref'].append(refs[i])
            data['word'].append(word[0])
            data['lex'].append(lex[0])
            data['morph'].append(morph[0])
            data['suffix'].append(None)
            data['strong'].append(strong[0])
            data['suffix_strong'].append(None)
df = pd.DataFrame(data)

In [90]:
display(HTML(df.head(n=10).to_html(index=False)))

ref,word,lex,morph,strong,suffix,suffix_strong
Gen.1.1-01,בְּ,ב,HR,H9003,None,None
Gen.1.1-01,רֵאשִׁ֖ית,רֵאשִׁית,Ncfsa,H7225,None,None
Gen.1.1-02,בָּרָ֣א,בָּרָא,HVqp3ms,H1254a,None,None
Gen.1.1-03,אֱלֹהִ֑ים,אֱלֹהִים,HNcmpa,H0430,None,None
Gen.1.1-04,אֵ֥ת,אֵת,HTo,H0853,None,None
Gen.1.1-05,הַ,ה,HTd,H9009,None,None
Gen.1.1-05,שָּׁמַ֖יִם,שָׁמַיִם,Ncmpa,H8064,None,None
Gen.1.1-06,וְ,ו,HC,H9002,None,None
Gen.1.1-06,אֵ֥ת,אֵת,To,H0853,None,None
Gen.1.1-07,הָ,ה,HTd,H9009,None,None


In [56]:
print(len(df.index))

424014


In [99]:
# https://gist.github.com/yakovsh/345a71d841871cc3d375
# Convert pointed text to consonantal. 
from unicodedata import normalize, combining
def norm(w):
    normalized = normalize('NFKD', w) # Reduce hebrew vowel ניקוד marks
    flattened = ''.join([c for c in normalized if not combining(c)])
    return flattened

In [ ]:
for i in range(1715, 1810):
    print(i, lex1[i], lex2[i+18])

In [ ]:
lex1 = [i for i in df['lex']]
lex2 = [i for i in OHB_ALIGNED['lex']]
words2 = [i for i in OHB_ALIGNED['word']]
k = 0
x = 0
count = 0
print(len(lex1))
for i, l in enumerate(lex1):
    j = i+k
    z = i+x
    l = norm(lex1[z]) if l else ''
    l2 = norm(lex2[j]) if lex2[j] else ''
    if type(words2[j]) != str:
        print('XX', i, l, l2)
        k += 1
    elif l[:2] != l2[:2]:
        if l2 in ['למה', 'לכן']:
            x += 1
        else:
            count += 1
            print(i, lex1[z-3:z+3], '\n', lex2[j-3:j+3])
            print(l, l2)
    if i % 10000 == 0:
        print(i, l, l2)

In [57]:
nan = 0
emp = 0
for i in words:
    if i == '':
        emp += 1
    if type(i) != str:
        nan +=1
print(emp, nan)

0 15


In [58]:
file = '../data_files/Combined/ohb_combined_aligned.csv'
OHB_ALIGNED = pd.read_csv(file, sep=',', low_memory=False)

In [ ]:
i = 1380
display(HTML(TBESH_DF.loc[i-5:i+5].to_html(index=False)))

In [ ]:
i = 2012
display(HTML(df.loc[i-5:i+5].to_html(index=False)))

In [ ]:
i = 1755
display(HTML(OHB_ALIGNED.loc[i-5:i+5].to_html(index=False)))

In [ ]:
words = [i for i in df['word']]
words2 = [i for i in OHB_ALIGNED['word']]
count = 0
for i, w in enumerate(words2):
    if w != words[i]:
        print(i, w, '\n', words[i])
        count += 1
    if count > 20:
        break